# Team Project - OCDM

## Explanation 
In this document is the code written to scrape the data from vivino, to look at if price have influence on the ratings of the different wines.

### Code
We will use selenium and Beautifull Soup in this document

#### Install the packages & librabies 

Explanation what we are doing:

In [18]:
#installing packages

!pip3 install selenium
!pip3 install webdriver_manager

#libraries

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager

service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)

# Opening the 'Vivino' website
url = "https://www.vivino.com/explore?e=eJzLLbI1VMvNzLM1UMtNrLA1NTBQS6609fNRS7Z1DQ1SKwDKpqfZliUWZaaWJOao5Rel2KakFier5SdV2ialFpfEF2QmZxerlZdEx9oamQD1FgMZqcVqYBIAh0UfNg%3D%3D"
driver.get(url)

# Optional: Adding some wait time for the page to fully load if needed
driver.implicitly_wait(10)  # 10 seconds


Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [19]:

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager

service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)

# Opening the 'Vivino' website
url = "https://www.vivino.com/explore?e=eJzLLbI1VMvNzLM1UMtNrLA1NTBQS6609fNRS7Z1DQ1SKwDKpqfZliUWZaaWJOao5Rel2KakFier5SdV2hYllmTmpRfHJ-eX5pUAAFlgGO4%3D"
driver.get(url)

# Optional: Adding some wait time for the page to fully load if needed
driver.implicitly_wait(10)  # 10 seconds

KeyboardInterrupt: 

In [20]:
try:
    # Create a function to click away the cookies
    accept_cookies_button = driver.find_element(By.ID,"didomi-notice-agree-button")
    accept_cookies_button.click()
    print("Cookies accepted.")
except Exception as e:
    print("Cookie acceptance button not found or could not be clicked:", e)


Cookies accepted.


In [21]:
# import requests
# import from bs4 import BeautifulSoup

import requests
from bs4 import BeautifulSoup

Infinite scroll to load all content (from the URL entered in the first cell)

In [22]:
from bs4 import BeautifulSoup
import time
driver.implicitly_wait(10)

# Infinite scroll to load more content
scroll_pause_time = 5  # Adjust if necessary
last_height = driver.execute_script("return document.body.scrollHeight")

while True:
    # Scroll down to the bottom of the page
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    
    # Wait for the new page to load
    time.sleep(scroll_pause_time)
    
    # Calculate new scroll height and compare with the last scroll height
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break  # Stop if we've reached the end of the page
    last_height = new_height

# Get the final page source after all content is loaded
page_source = driver.page_source

# Parse the page source with BeautifulSoup
soup = BeautifulSoup(page_source, 'html.parser')

Scraping all the data

In [23]:
import csv
from bs4 import BeautifulSoup

# Create empty lists to store the data
brands = []
wines = []
ratings = []
prices = []

# Find all wine entries on the page
wine_entries = soup.find_all(class_='card__card--2R5Wh wineCard__wineCardContent--3cwZt')

for entry in wine_entries:
    # Extract brand
    brand = entry.find(class_='wineInfoVintage__truncate--3QAtw')
    if brand:
        brands.append(brand.get_text(strip=True))

    # Extract wine name
    wine = entry.find(class_='wineInfoVintage__vintage--VvWlU wineInfoVintage__truncate--3QAtw') 
    if wine:
        wines.append(wine.get_text(strip=True))

    # Extract rating
    rating = entry.find(class_='vivinoRating_averageValue__uDdPM')
    if rating:
        ratings.append(rating.get_text(strip=True))

    # Extract curreny & price
    price_divs = entry.find_all( class_='addToCartButton__price--qJdh4') 
    for price_div in price_divs:
        currency = price_div.find('div', class_='addToCartButton__currency--2CTNX')  
        price = price_div.find_all('div')[1]  
        full_price = f"{currency.get_text(strip=True) if currency else ''}{price.get_text(strip=True) if price else ''}"
        prices.append(full_price)  # Save the full price to the list

Saving the data to a CSV file

In [24]:
with open('wine_data.csv', mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)

    # Write the header row
    writer.writerow(['Brand', 'Wine', 'Rating', 'Price'])

    # Write the data rows
    for brand, wine, rating, price in zip(brands, wines, ratings, prices):
        writer.writerow([
            brand,
            wine,
            rating,
            price
        ])

print("Data saved to 'wine_data.csv' successfully.")


Data saved to 'wine_data.csv' successfully.


ROOD 0-20

In [ ]:
## Rode Spaanse Wijnen 0-20 euro

url = 'https://www.vivino.com/explore?e=eJwdijELgCAUBv_NN1v7G9uiIWiKCHtZSJnik8p_n7TcDXcuUgVnL1Jw-qVagTN1LZiaoUcocd_o1tGapE_4uNJqhOGXTIuRNAfLh-BJ41RWzlJsBD8_VgEezg%3D%3D'
request = requests.get(url, headers =  {'User-agent': 'Mozilla/5.0'})
red_0_20 = BeautifulSoup(request.text)
red_0_20

ROOD 20-500

In [ ]:
## Rode Spaanse Wijnen 20-500 euro

url = 'https://www.vivino.com/explore?e=eJwdir0KgCAURt_mmy1ovGNbNARNEWE3EyE11H58-6TlnOEcG6iCNY5qAStfaoQAZ-o7MLXjgLNkvdMtg1FJHvBho01Fhl8zBZmM03Fhf7mEJ01zuTnHYhXx8wPloSCM'
request = requests.get(url, headers =  {'User-agent': 'Mozilla/5.0'})
red_20_500 = BeautifulSoup(request.text)
red_20_500

WIT 0-20

In [ ]:
## Witte Spaanse Wijnen 0-20 euro

url = 'https://www.vivino.com/explore?e=eJwdi7EOQDAYBt_mm7F_o00MEpOIVJU0VKV_g769xnI3XM4FlnD2ZAGnXlYFdGLbQLPuO1w5bitvFayJ6oAPCxcjGn5OnI3E6bJ6FzxxGFnlVbKN4OcHVhEezw%3D%3D'
request = requests.get(url, headers =  {'User-agent': 'Mozilla/5.0'})
white_0_20 = BeautifulSoup(request.text)
white_0_20 

WIT 20-500

In [ ]:
## Witte Spaanse Wijnen 21-500 euro

url = 'https://www.vivino.com/explore?e=eJzLLbI1VMvNzLM1MlDLTaywNTUwUEuutPXzUUu2dQ0NUisASqen2ZYlFmWmliTmqOUXpdimpBYnq-UnVdompRaXxBdkJmcXq5WXRMfaGgG1FgPp1GI1MAkAfZUfNA%3D%3D'
request = requests.get(url, headers =  {'User-agent': 'Mozilla/5.0'})
white_20_500 = BeautifulSoup(request.text)
white_20_500 

ROSE 0-500

In [ ]:
## Rose Spaanse Wijnen 0-500 euro

url= 'https://www.vivino.com/explore?e=eJwdi70KgCAABt_mmw1q_Ma2aAiaIsLMQkgNtb-3T1ruhuNsYAFrHAWsfFgJAfWybaBY9x2OXLeVlwxGJ7nDh4WLjgp-fhlkMm6Lk_KnS7jTMLLMd8zWET8_0d4gXQ%3D%3D'
request = requests.get(url, headers =  {'User-agent': 'Mozilla/5.0'})
rose_0_500 = BeautifulSoup(request.text)
rose_0_500

SPARKLING 0-500

In [ ]:
## Sparkling Spaanse Wijnen 0-500 euro

url = 'https://www.vivino.com/explore?e=eJwdi70KgCAABt_mm41o_Ma2aAiaIsLMQkgNtb-3T1ruhuNsYAFrHAWsfFgJAfWybaBY9x2OXLeVlwxGJ7nDh4WLjgp-fhlkMm6Lk_KnS7jTMLLMd8zWET8_0c4gXA%3D%3D'
request = requests.get(url, headers =  {'User-agent': 'Mozilla/5.0'})
sparkling_0_500 = BeautifulSoup(request.text)
sparkling_0_500

DESSERT 0-500

In [ ]:
## Dessert Spaanse Wijnen 0-500 euro

url = 'https://www.vivino.com/explore?e=eJwdi70KgCAABt_mm22Ipm9si4agKSLMLITUUPt7-6TlbjjOBhawxlHAyoelEFAv2waKdd_hyHVbeclgdJI7fFi46Kjg55dBJuO2OCl_uoQ7DSOrfMdsHfHzA9IOIGA%3D'
request = requests.get(url, headers =  {'User-agent': 'Mozilla/5.0'})
dessert_0_500 = BeautifulSoup(request.text)
dessert_0_500

FORTIFIED 0-500

In [ ]:
## Fortified Spaanse Wijnen 0-500 euro

url = 'https://www.vivino.com/explore?e=eJzLLbI1VMvNzLM1UMtNrLA1NTBQS6609fNRS7Z1DQ1SKwDKpqfZliUWZaaWJOao5Rel2KakFier5SdV2ialFpfEF2QmZxerlZdEx9oamQD1FgMZqcVqYBIAh0UfNg%3D%3D'
request = requests.get(url, headers =  {'User-agent': 'Mozilla/5.0'})
fortified_0_500 = BeautifulSoup(request.text)
fortified_0_500